In [ ]:
!pip install spotipy
!pip install lyricsgenius

     |████████████████████████████████| 62 kB 929 kB/s 
     |████████████████████████████████| 138 kB 67.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 59 kB 4.3 MB/s 


In [ ]:
import os
import re
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer

import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import random
from random import randrange

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

genius_key = "ZFekHtccXAr1BS_AIMStDXgCLhV1D6or7-sm1oDxitaWp_Dxaq1eTQ6MI340rdmy"
client_id = "4b98fe888b204babb61c9358f4555b61"
client_secret = "ce07bcdc67c44f5fa9640bb27356f0ee"

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
songs_df = pd.read_csv("/content/drive/MyDrive/data.csv")
songs_df['artists'] = [" ".join(re.findall("[a-zA-Z]+", i)) for i in songs_df['artists']]

lyrics_df = pd.read_csv("/content/drive/MyDrive/lyrics_df.csv")

print("Shape of songs data", songs_df.shape)
print("Shape of user playlist data", user_df.shape)
print("Shape of lyrics data", lyrics_df.shape)

Shape of songs data (174389, 19)
Shape of user playlist data (2682257, 26)
Shape of lyrics data (174389, 5)


In [ ]:
songs_df = songs_df.sort_values(by=['name'], ascending=[True]).groupby(['name']).head(1).sort_index().reset_index(drop=True)

In [ ]:
songs_df.head()

acousticness                      artists  ...  valence  year
0      0.991000                  Mamie Smith  ...   0.6340  1920
1      0.993000                  Mamie Smith  ...   0.6890  1920
2      0.000173              Oscar Velazquez  ...   0.0422  1920
3      0.295000                         Mixe  ...   0.2990  1920
4      0.996000  Mamie Smith Her Jazz Hounds  ...   0.4770  1920

[5 rows x 19 columns]

In [ ]:
"""
User playlist Feature Engineering
"""

playlist = pd.read_csv("/content/drive/MyDrive/spotify_dataset.csv", error_bad_lines=False)

playlist.columns = ['user_id','artistname','trackname','playlistname']
user_df = playlist[playlist['trackname'].isin(songs_df['name'])]
print(user_df.shape)

user_df = pd.merge(user_df,songs_df,left_on = "trackname", right_on= "name", how = "left")

# User features
print("Unique Users: ",user_df.user_id.nunique())

gender = ["M"]*9768+ ["F"]*5767
random.shuffle(gender)
age = [random.randint(11,69) for i in range(0,15535)]
loc = ["US"]*4987 + ["UK"]*3756 + ["CA"]*2200 + ["IN"]*2100 + ["SG"]*2001 + ["NZ"]*491
random.shuffle(loc)

feat_df = pd.DataFrame({"user_id":list(user_df.user_id.unique()),'gender':gender,"age":age,"country":loc})
user_df = pd.merge(user_df, feat_df, on = "user_id", how = "left")

b'Skipping line 14735: expected 4 fields, saw 5\nSkipping line 15931: expected 4 fields, saw 6\nSkipping line 18825: expected 4 fields, saw 5\nSkipping line 22437: expected 4 fields, saw 6\nSkipping line 23009: expected 4 fields, saw 7\nSkipping line 24244: expected 4 fields, saw 6\nSkipping line 24245: expected 4 fields, saw 6\nSkipping line 24247: expected 4 fields, saw 6\nSkipping line 24248: expected 4 fields, saw 6\nSkipping line 24250: expected 4 fields, saw 6\nSkipping line 24253: expected 4 fields, saw 6\nSkipping line 24254: expected 4 fields, saw 5\nSkipping line 24467: expected 4 fields, saw 5\nSkipping line 25091: expected 4 fields, saw 5\nSkipping line 28223: expected 4 fields, saw 5\nSkipping line 28506: expected 4 fields, saw 5\nSkipping line 28512: expected 4 fields, saw 5\nSkipping line 28513: expected 4 fields, saw 5\nSkipping line 28520: expected 4 fields, saw 5\nSkipping line 32192: expected 4 fields, saw 5\nSkipping line 35773: expected 4 fields, saw 5\nSkipping li

(4441715, 4)
Unique Users:  15535


In [ ]:
user_df.user_id.nunique()

15535

In [ ]:
user_df.to_csv("/content/drive/MyDrive/user_playlist_data.csv",index=False)

In [ ]:
songs_df.to_csv("/content/drive/MyDrive/spotify_song_data_v1.csv",index=False)

In [ ]:
songs_df = pd.read_csv("/content/drive/MyDrive/data.csv")
songs_df['artists'] = [" ".join(re.findall("[a-zA-Z]+", i)) for i in songs_df['artists']]

lyrics_df = pd.read_csv("/content/drive/MyDrive/lyrics_df.csv")
user_df = pd.read_csv("/content/drive/MyDrive/user_playlist_data.csv")

print("Shape of songs data", songs_df.shape)
print("Shape of user playlist data", user_df.shape)
print("Shape of lyrics data", lyrics_df.shape)

Shape of songs data (174389, 19)
Shape of user playlist data (2682257, 26)
Shape of lyrics data (174389, 5)
